# Do not store credentials and ID's of any kind in version control!

# Retrieve STATEMENT and CALLSTACK from SYSTRACETABLESQL records

Also see: [https://docs.sqlalchemy.org/en/latest/dialects/mssql.html](https://docs.sqlalchemy.org/en/latest/dialects/mssql.html)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import urllib
import requests
import json
from openpyxl import load_workbook
from openpyxl.utils.cell import rows_from_range
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.table import Table, TableStyleInfo

# Define function to extract fields from SYSTRACETABLESQL in SQL Server

In [ ]:
def get_ax_trace_info( recid, db, trace_table = 'SYSTRACETABLESQL'):
    params = urllib.parse.quote_plus(f"DRIVER={{SQL Server Native Client 11.0}};"
                                 f"SERVER=localhost;"
                                 f"DATABASE={db};"
                                 f"Trusted_Connection=yes")
    # Connect via current Windows credentials
    conn = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
    
    df = pd.read_sql(f"SELECT TOP 1 STATEMENT, CALLSTACK FROM {trace_table} WHERE RECID = {recid}", conn)
    
    # Convert multi line CALLSTACK into list while dropping empty line (at the end)
    if df.loc[0, 'CALLSTACK'] is not None:
        callstack_list = df.loc[0, 'CALLSTACK'].split('\n')
        callstack_list = [x for x in callstack_list if len(x) > 0]
    else:
        callstack_list = ['']
        
    # let sql-format.com format STATEMENT and convert (multi-line) result into list
    payload_dict = {'text': df.loc[0, 'STATEMENT'], 'options': '{}', 'caretPosition[x]': 0, 'caretPosition[y]': 0, 'saveHistory': 'true' }
    r = requests.post("http://sql-format.com", data=payload_dict)
    formatted_sql = json.loads(r.text)
    # Convert formatted sql to list
    formatted_sql_list = formatted_sql['Text'].split('\r\n')
    # If error then add error message to list
    if formatted_sql['ErrorInfo']:
        formatted_sql_list.append(formatted_sql['ErrorInfo']['ErrorMessage'])
    
    # Return dict with statement and call stack
    return {
        'statement': pd.DataFrame(data={'SQL': formatted_sql_list }),
        'callstack': pd.DataFrame(data={'Call stack': callstack_list }),
    }
    

In [ ]:
dict = get_ax_trace_info( recid = 5637152900, db = 'METADATADB')

In [ ]:
# dict[ 'statement']

In [ ]:
# dict[ 'callstack']

In [ ]:
# Process RecIds in workbook "Long running queries analyse.xlsx" 

In [ ]:
xlsx_file = 'Workbook-containing-range-named-RecIds.xlsx'

In [ ]:
wb = load_workbook(xlsx_file)
RecIds_range = wb.defined_names['RecIds']

In [ ]:
dests = RecIds_range.destinations # returns a generator of (worksheet title, cell range) tuples

for title, coord in dests:
    print( title, coord )
    ws = wb[title]
    for address in rows_from_range(coord):
        print(address)
        recid = ws[address[0]].value
        print(recid)
        trace_info_dict = get_ax_trace_info( recid = recid, db = 'METADATADB')
        #print(trace_info_dict['callstack'])
        new_ws = wb.create_sheet(f"{recid}")
        concat_df = pd.concat([trace_info_dict['statement'], trace_info_dict['callstack']], axis=1, ignore_index=True)
        concat_df.columns = ['SQL', 'Callstack']
        for r in dataframe_to_rows(concat_df, index=False, header=True):
            new_ws.append(r)
        
        tab = Table(displayName=f"Table{recid}", ref=new_ws.calculate_dimension())
        # Add a default style with striped rows and banded columns
        style = TableStyleInfo(name="TableStyleMedium2", showFirstColumn=False,
            showLastColumn=False, showRowStripes=True, showColumnStripes=False)
        tab.tableStyleInfo = style
        new_ws.add_table(tab)

        # Resize columns
        dims = {}
        for row in new_ws.rows:
            for cell in row:
                if cell.value:
                    dims[cell.column] = max((dims.get(cell.column, 0), len(str(cell.value))))    
        for col, value in dims.items():
            new_ws.column_dimensions[col].width = value
            
    print('Done with for loop!')

wb.save(xlsx_file)        
wb.close()     